# Reproduce Transformer from Attention is All You Need

## Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
from dataset_automl import Dataset
from utils import NUM_PROC, DEVICE, free_memory
from model import TransformerModel
from transformer_automl import Transformer


print("Number of processors: ", NUM_PROC)
print("Device: ", DEVICE)

Number of processors:  32
Device:  cuda


## AutoML

In [2]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
tokenizer("[PAD]")

{'input_ids': [101, 0, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

## Transformer Lite from Scratch

Using half the dimension as the base model: $d_{\rm model} = 256$, $d_{\rm ff} = 1024$. 

### Tokenizer

Byte-Pair Encoding with shared (English + German) vocabulary of 37000 tokens.

In [2]:
tokenizer = get_tokenizer(name="wmt14", language="de-en", vocab_size=37000)

Loaded tokenizer from ../tokenizer-wmt14-de-en.json


### Dataset

The dataset is downloaded at ~/.cache/huggingface/datasets/. I've turned off dataset caching to avoid disk explosion.

In [3]:
dataset = Dataset(name="wmt14", language="de-en", percentage=1)

In [4]:
dataset.tokenize(tokenizer)

Map (num_proc=32):   0%|          | 0/45088 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/3000 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/3003 [00:00<?, ? examples/s]

In [5]:
dataloader = {}
for split in ["train", "validation", "test"]:
    dataloader[split] = dataset.get_dataloader(split=split, batch_size=64, shuffle=True, min_len=1, max_len=128)


Filter:   0%|          | 0/45088 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/44871 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/2996 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3003 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/3003 [00:00<?, ? examples/s]

### Train

In [7]:
# create the transformer model
model = TransformerModel(vocab_size=len(tokenizer.get_vocab()), d_model=256, dim_feedforward=1024).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=512**-0.5, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda nstep: min((nstep + 1) ** -0.5, (nstep + 1) * 4000 ** -1.5))
loss_fn = nn.CrossEntropyLoss() # could add label smoothing

In [7]:
# load model
# model.load_state_dict(torch.load("model_1.pth"))

In [8]:
# free_memory("model")
free_memory()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 256754 KiB | 256754 KiB | 256754 KiB |      0 B   |
|       from large pool | 212992 KiB | 212992 KiB | 212992 KiB |      0 B   |
|       from small pool |  43762 KiB |  43762 KiB |  43762 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 256754 KiB | 256754 KiB | 256754 KiB |      0 B   |
|       from large pool | 212992 KiB | 212992 KiB | 212992 KiB |

In [9]:
# create the transformer wrapper
transformer = Transformer(model, tokenizer)

In [10]:
transformer.train(dataloader, model, loss_fn, optimizer, scheduler)

-------------------------------
Epoch 1/1
Accuracy: 0.0%, Avg loss: 11.689628  [   64/44871]  [0:00:13 < 2:33:29]


KeyboardInterrupt: 

### Evaluate

In [72]:
sample = dataset.dataset["test"]["translation"][5]
transformer.predict(sample["de"], sample["en"])

Accuracy: 26.1%
The
The same
The Kl s
The Kl user and
The Kl user lights and
The Kl user lights protect the
The Kl user lights protect cycl and
The Kl user lights protect cycl ists and
The Kl user lights protect cycl ists , and
The Kl user lights protect cycl ists , as well
The Kl user lights protect cycl ists , as well as
The Kl user lights protect cycl ists , as well as the
The Kl user lights protect cycl ists , as well as those who
The Kl user lights protect cycl ists , as well as those travelling and
The Kl user lights protect cycl ists , as well as those travelling by the
The Kl user lights protect cycl ists , as well as those travelling by bus h
The Kl user lights protect cycl ists , as well as those travelling by bus and the
The Kl user lights protect cycl ists , as well as those travelling by bus and the other
The Kl user lights protect cycl ists , as well as those travelling by bus and the residents .
The Kl user lights protect cycl ists , as well as those travelling by bus an

In [73]:
print(transformer.translate("Ich bin ein Berliner."))

I have a pleasure to make a clear distinction .


In [74]:
for i in range(5):
    samples = dataset.dataset["test"]["translation"]
    idx = np.random.randint(len(samples))
    sample = samples[i]
    print(f"#{i+1}")
    print(f"Source: {sample['de']}")
    print(f"Target: {sample['en']}")
    print(f"Prediction: {transformer.translate(sample['de'])}")
    print()

#1
Source: Gutach: Noch mehr Sicherheit für Fußgänger
Target: Gutach: Increased safety for pedestrians
Prediction: The Council is also a more important point .

#2
Source: Sie stehen keine 100 Meter voneinander entfernt: Am Dienstag ist in Gutach die neue B 33-Fußgängerampel am Dorfparkplatz in Betrieb genommen worden - in Sichtweite der älteren Rathausampel.
Target: They are not even 100 metres apart: On Tuesday, the new B 33 pedestrian lights in Dorfparkplatz in Gutach became operational - within view of the existing Town Hall traffic lights.
Prediction: You are not going to have the same resources , which you know : ' You are not being used in the new form of the new waste - in the new Sea .

#3
Source: Zwei Anlagen so nah beieinander: Absicht oder Schildbürgerstreich?
Target: Two sets of lights so close to one another: intentional or just a silly error?
Prediction: Thirdly , what are the same kind of or whether or not they are they going to be a good example ?

#4
Source: Diese Fra

## DEBUG

In [14]:
for name in ["src_len", "tgt_len"]:
    len_list = dataset.dataset["train"][name]
    tot = sum(len_list)
    count = 0
    for num in len_list:
        if num <= 256:
            count += num
    print(f"count: {count}, tot: {tot}, percentage: {count/tot*100:.2f}%")

count: 14302156, tot: 14303264, percentage: 99.99%
count: 14339828, tot: 14340777, percentage: 99.99%


In [27]:
for batch in dataloader["train"]:
    x, x_mask, y, y_mask = batch.values()
    print(x.shape, x_mask.shape, y.shape, y_mask.shape)
    x = model.embedding(x.to(DEVICE))
    print(x.shape)
    break

torch.Size([64, 128]) torch.Size([64, 128]) torch.Size([64, 128]) torch.Size([64, 128])
torch.Size([64, 128, 256])
